## Repeated measures ANOVA validation using pyvttlb & R

In [1]:
from __future__ import division

import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats
import matplotlib.pylab as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats
import statsmodels.sandbox.stats as smsb
import rpy2
import warnings
import itertools
import pyvttbl as pt
%load_ext rpy2.ipython

/home/orcasha/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


### Create dataset from https://m-clark.github.io/docs/mixedModels/anovamixed.html

In [2]:
%R library(dplyr)
%R library(lmPerm)
%R treat  = rep(c('treat', 'control'), e=5)
%R pre = c(20,10,60,20,10,50,10,40,20,10)
%R post = c(70,50,90,60,50,20,10,30,50,10)

%R df = data.frame(id=factor(1:10), treat, pre, post)
%R change = post-pre

%R dflong = tidyr::gather(df, key=time, value=score, pre:post) %>% arrange(id);

/home/orcasha/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/home/orcasha/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/home/orcasha/anaconda2/lib/python2.7/site-packages/rpy2/rinterface/__init__.py:186: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)


### Now the same in python using dataframes

In [3]:
df = pd.melt(pd.DataFrame({'treat': ['treat'] * 5 + ['control'] * 5, 
        'pre': [20,10,60,20,10,50,10,40,20,10], 
        'post': [70,50,90,60,50,20,10,30,50,10], 'id': range(1,11)}), 
             id_vars = ['treat', 'id'], 
             var_name = 'time',
             value_vars = ['pre', 'post'], 
             value_name = 'score')

## Fit repeated measures using ID as the random variable

In [4]:
%R anovaModelRM = aov(score ~ treat*time + Error(id), dflong)
%R summary <- summary(anovaModelRM)
%Rpull summary
print(summary)


Error: id
          Df Sum Sq Mean Sq F value Pr(>F)
treat      1   1805    1805   3.406  0.102
Residuals  8   4240     530               

Error: Within
           Df Sum Sq Mean Sq F value  Pr(>F)   
time        1   1805    1805   13.88 0.00582 **
treat:time  1   2205    2205   16.96 0.00335 **
Residuals   8   1040     130                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



## Do the same using pyvttlb

In [5]:
x = pt.DataFrame(df)
aov = x.anova('score', sub = 'id', wfactors = ['time'], bfactors = ['treat'])
print(aov)

score ~ time * treat

TESTS OF BETWEEN-SUBJECTS EFFECTS

Measure: score
     Source        Type III   df    MS      F     Sig.    et2_G   Obs.     SE     95% CI   lambda   Obs.  
                      SS                                                                            Power 
Between Subjects       6045    9                                                                          
treat                  1805    1   1805   3.406   0.102   0.255      5   12.113   23.742    2.129   0.251 
Error                  4240    8    530                                                                   

TESTS OF WITHIN SUBJECTS EFFECTS

Measure: score
   Source                           Type III   eps   df    MS      F      Sig.    et2_G   Obs.    SE     95% CI   lambda   Obs.  
                                       SS                                                                                  Power 
time           Sphericity Assumed       1805     -    1   1805   13.885   0.006   0

/home/orcasha/anaconda2/lib/python2.7/site-packages/pyvttbl/stats/_anova.py:1240: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  return list(array(list(zeros((p-len(b))))+b)+1.)


In [6]:
#Extracting data from the object

print(aov.keys())

[('time',), ('treat',), ('time', 'treat'), ('id',), ('TOTAL',), ('WITHIN',), ('time', 'id')]


In [7]:
#Effects can be accessed from the tuple using indexing (eg. 0 = time, 1 = treat, 2 = interaction)
print(aov.items())[0]

(('time',), {'power_gg': 0.95254063239451958, 'ci': 8.3144161502948037, 'critT': 2.3060041350333709, 'F_lb': 13.884615384615385, 'df_gg': 1.0, 'F_hf': 13.884615384615385, 'se_gg': 4.242049056272859, 'se_hf': 4.242049056272859, 'y2': array([ 39.25,  29.75]), 'eps_lb': 1.0, 'lambda_gg': 17.35576923076923, 'se_lb': 4.242049056272859, 'df_hf': 1.0, 'dfe_hf': 8.0, 'ci_lb': 8.3144161502948037, 'eps_gg': 1.0, 'df_lb': 1.0, 'mss': 1805.0, 'p_hf': 0.0058193283883128761, 'power_lb': 0.95254063239451958, 'critT_hf': 2.3060041350333709, 'lambda_hf': 17.35576923076923, 'mse_lb': 130.0, 'dfe': 8.0, 'obs_gg': 10.0, 'mse': 130.0, 'mse_hf': 130.0, 'critT_gg': 2.3060041350333709, 'power': 0.95254063239451958, 'F': 13.884615384615385, 'df': 1.0, 'ci_gg': 8.3144161502948037, 'mse_gg': 130.0, 'mss_gg': 1805.0, 'dfe_lb': 8.0, 'critT_lb': 2.3060041350333709, 'sse': 1040.0, 'p_lb': 0.0058193283883128761, 'power_hf': 0.95254063239451958, 'F_gg': 13.884615384615385, 'eps_hf': 1.0, 'dfe_gg': 8.0, 'ss': 1805.0, '

In [8]:
#Specific items within tuple: 0 = effect name, 1 = dictionary (then access as a normal dict)
print(aov.items())[0][1]['F']

13.8846153846


## Summary:

ID effect on Treatment in R: F = 3.406, p = 0.102

ID effect on Treatment in Python: F = 3.406   p = 0.102

R adjusted effects:

| condition | Df | Sum Sq | Mean Sq | F value |  Pr(>F) |
| --------- | -- | ------ | ------- | ------- | ------- |
|time | 1 | 1805 | 1805 | 13.88 | 0.00582 | 
|treat:time | 1 | 2205 | 2205 | 16.96 | 0.00335 |

Python adjusted effects:

| condition | Df | Sum Sq | Mean Sq | F value |  Pr(>F) |
| --------- | -- | ------ | ------- | ------- | ------- |
| time | 1 | 1805 | 1805 | 13.885 | 0.006 |
| time * treat | 1 | 2205 | 2205 | 16.962 | 0.003 |


## Identical.





